In [ ]:
import os
import json
import pandas as pd

In [27]:
json_dir = '/home/morrissimons/Desktop/avancerad_ml/datasets'
merged_data = []

for filename in os.listdir(json_dir):
    if filename.endswith('.json') and filename != '6_dataset_full.json':
        file_path = os.path.join(json_dir, filename)
        with open(file_path, 'r') as file:
            data = json.load(file)
            if isinstance(data, list):
                merged_data.extend(data)  # Flatten list
            else:
                merged_data.append(data)

output_file = os.path.join(json_dir, '6_dataset_full.json')
with open(output_file, 'w') as file:
    json.dump(merged_data, file, indent=4)

In [29]:
# Convert merged_data list to a pandas DataFrame
df_merged = pd.DataFrame(merged_data)

df_merged.head()

,id,text,to_mask
0,1,Hej! Jag heter Anna Svensson och vill flytta m...,"[{'label': '[NAMN]', 'value': 'Anna Svensson'}..."
1,2,"Jag har nyligen flyttat till Storgatan 12, 123...","[{'label': '[ADRESS]', 'value': 'Storgatan 12,..."
2,3,Jag vill ändra mina kontaktuppgifter. Mitt nya...,"[{'label': '[TELEFONNUMMER]', 'value': '070-12..."
3,4,Jag försökte logga in med mitt passnummer AB12...,"[{'label': '[PASSNUMMER]', 'value': 'AB1234567..."
4,5,Jag behöver uppdatera mitt bankkontonummer. De...,"[{'label': '[BANKKONTONUMMER]', 'value': 'SE98..."


In [30]:
count_num_ids = df_merged["id"].nunique()
print(f"Number of unique ids: {count_num_ids}")

Number of unique ids: 700


In [45]:
# Count unique labels
count_num_labels = df_merged["to_mask"]

distinct_labels = (
    df["to_mask"]
    .explode()
    .apply(lambda x: x["label"] if isinstance(x, dict) else None)
    .dropna()
    .unique()
)
print(distinct_labels)

['[NAMN]' '[PERSONNUMMER]' '[ORG-NUMMER]' '[ADRESS]' '[ADRESSHISTORIA]'
 '[TELEFONNUMMER]' '[E-POST]' '[PASSNUMMER]' '[IP-ADRESS]'
 '[BANKKONTONUMMER]' '[ARBETSPLATS]' '[ANSTÄLLNINGSTID]' '[CIVILSTAND]'
 '[POSTNUMMER]' '[KREDITKORTNUMMER]' '[BANKGIRONUMMER]' '[ARBETSGIVENHET]'
 '[TITEL]' '[FÖDELSEDAG]' '[NATIONALITET]' '[FÖDELSEPLATS]' '[ETNICITET]'
 '[SOCIALA_MEDIER_PROFILER]' '[ÅLDER]' '[KÖN]' '[CLEARINGNUMMER]' '[IBAN]'
 '[BIC]' '[STAD]' '[MEDARBETARID]' '[RELIGION]' '[LÄN]'
 '[SKATTEREGISTRERINGSNR]' '[LAND_TILLDELAT_SKATTEREGISTRERINGSNR]'
 '[PLUSGIRO]' '[FÖRSÄKRINGSNUMMER]' '[FÖRSÄKRAD]' '[INBETALNINGSREFERENS]'
 '[UTDELNINGSADRESS]' '[AVTALSSOMRÅDE]' '[AVTALSNUMMER]'
 '[PENSIONSÅLDER_KONTAKTPERSON]' '[STYRELSENS_SÄTE]' '[TIN]'
 '[FÖDELSELAND]' '[LEI]' '[GIIN]' '[AKTIEINNEHAV]']
